In [ ]:
import syft as sy
import torch
from tools import models
import numpy as np
import pandas as pd
import time

sy.load('opacus')
np.random.seed(42) # The meaning of life!

#### Connect to to Data Owner 1

In [ ]:
duet1 = sy.launch_duet(loopback=True)

#### Connect to to Data Owner 2

In [ ]:
duet2 = sy.launch_duet(loopback=True)

#### Get the Pointers to the data

In [ ]:
# Data from owner 1
train_data_ptr_1 = duet1.store[0]
train1_labels_ptr_1 = duet1.store[1]

test_data_ptr_1 = duet1.store[2]
test_labels_ptr_1 = duet1.store[3]

# Data from owner 2
train_data_ptr_2 = duet2.store[0]
train_labels_ptr_2 = duet2.store[1]

test_data_ptr_2 = duet2.store[2]
test_labels_ptr_2 = duet2.store[3]

#### Setup the Models and Differential Privacy Engine

In [ ]:
 Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = True # Whether or not this run should be tracked in the results csv file
DP = False # Whether or not Differential Privacy should be applied

# Parameters for training and Differential Privacy
BATCH_SIZE = 100
EPOCHS = 10
LEARNING_RATE = 0.003 if DP else 0.0005

DELTA = 1e-3 # Set to be less then the inverse of the size of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)
NOISE_MULTIPLIER = 2.0 # The amount of noise sampled and added to the average of the gradients in a batch (from https://opacus.ai/tutorials/building_image_classifier)
MAX_GRAD_NORM = 1.2 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step (from https://opacus.ai/tutorials/building_image_classifier)

length = len(train_data_ptr)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

# Getting remote and local instances (DO1)
local_model_1 = models.Deep2DNet(torch)
remote_model_1 = local_model_1.send(duet1)
remote_torch_1 = duet1.torch
remote_opacus_1 = duet1.opacus

# Getting remote and local instances (DO2)
local_model_2 = models.Deep2DNet(torch)
remote_model_2 = local_model_2.send(duet2)
remote_torch_2 = duet2.torch
remote_opacus_2 = duet2.opacus

# Setting device to train on (DO1)
cuda_available_1 = remote_torch_1.cuda.is_available().get(request_block=True, reason='Need to check for available GPU!')
if cuda_available_1:
    device_1 = remote_torch_1.device('cuda:0')
    remote_model_1.cuda(device)
else:
    device_1 = remote_torch_1.device('cpu')
    remote_model_1.cpu()
    
# Setting device to train on (DO2)
cuda_available_2 = remote_torch_2.cuda.is_available().get(request_block=True, reason='Need to check for available GPU!')
if cuda_available_2:
    device_2 = remote_torch_2.device('cuda:0')
    remote_model_2.cuda(device)
else:
    device_2 = remote_torch_2.device('cpu')
    remote_model_2.cpu()

# Optimizer and Loss Function (DO1)
params_1 = remote_model_1.parameters()
optim_1 = remote_torch_1.optim.Adam(params=params_1, lr=LEARNING_RATE) 
loss_function_1 = remote_torch_1.nn.CrossEntropyLoss()

# Optimizer and Loss Function (DO2)
params_2 = remote_model_2.parameters()
optim_2 = remote_torch_2.optim.Adam(params=params_1, lr=LEARNING_RATE) 
loss_function_2 = remote_torch_2.nn.CrossEntropyLoss()

# Setting up Differential Privacy Engine
if DP:
    privacy_engine_ptr_1 = remote_opacus.privacy_engine.PrivacyEngine(
        remote_model_1.real_module, sample_rate=SAMPLE_RATE,
        noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
    )
    privacy_engine_ptr_2.attach(optim_1)
    
    privacy_engine_ptr_2 = remote_opacus.privacy_engine.PrivacyEngine(
        remote_model_2.real_module, sample_rate=SAMPLE_RATE,
        noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
    )
    privacy_engine_ptr_2.attach(optim_2)
else:
    privacy_engine_ptr_1 = None
    privacy_engine_ptr_1 = None

#### Train the Models at each Data Owner